## AWS  Trascribe

In [ ]:
!pip install sagemaker

In [29]:
import boto3 
import pandas as pd
import boto3                                  #https://boto3.amazonaws.com/v1/documentation/api/latest/index.html
import sagemaker
from datetime import datetime
import time
import json
region = boto3.session.Session().region_name
sess = sagemaker.session.Session()            #https://boto3.amazonaws.com/v1/documentation/api/latest/reference/core/session.html

#### I could not run this part of the code. However, I could  run the rest of codes without this part. 

In [32]:
# from sagemaker import get_execution_role

# role = get_execution_role()
# print(role)

#### setting `

In [33]:
bucket = 'vnsny-research-transcribeinput-test'  # replace your s3 bucket name
input_folder = 'amazon-grant/amazon_grant-piolot-test'  # replace the folder name where you put audio files in s3 bucket
output_folder = 'vnsny-research-transcribeoutput-test'  # replace the folder name where you want Transcribe to output in your s3 bucket

In [34]:

# #role = sagemaker.get_execution_role()

# bucket = 'columbia-univ-data-transcribe'  # replace your s3 bucket name
# input_folder = 'raw-audio'  # replace the folder name where you put audio files in s3 bucket
# output_folder = 'vnsny-research-transcribeoutput-test'  # replace the folder name where you want Transcribe to output in your s3 bucket

In [35]:
s3 = boto3.client('s3')

s3_response = s3.list_objects_v2(           #Returns some or all (up to 1,000) of the objects in a bucket with each request
    Bucket = bucket,
    Prefix = f'{input_folder}/'
)

file = []
for i in range(len(s3_response['Contents'])-1):
    file.append(s3_response['Contents'][i+1]['Key'].split('/')[-1])
print(file)

['IPOD_HG_40921.mp4', 'IPOD_RN_04092021.mp4', 'IPOD_HG_40921.mp4', 'SONY_HG_04092021.MP3', 'SONY_LB_21721.MP3', 'SONY_TL_21621.MP3', 'SONY_TL_21821.MP3', 'SONY_withMirohphone.mp3', 'dual_channel_recording.m4a', 'SONY_MH_21921.MP3', 'SONY_RB_21921(2).MP3', 'SONY_TLA_22321.MP3', 'VOX_TLA_22321 copy.MP3']


## Start Transcribe job

#### Sherry's code 

In [36]:
# transcribe = boto3.client('transcribe')
# all_jobs = []

# for i in range(len(file)):
#     now = datetime.now()
#     time_now = now.strftime('%H.%M.%S')
#     job_name = f'transcribe-jid-{file[i]}-{time_now}'
#     all_jobs.append(job_name)
#     media_uri = f's3://{bucket}/{input_folder}/{file[i]}'
#     transcribe.start_transcription_job(
#         TranscriptionJobName = job_name,
#         MediaFormat = 'wav',
#         Media = {'MediaFileUri': media_uri},
#         LanguageCode = 'en-US',
#         OutputBucketName = bucket,
#         OutputKey = f'{output_folder}/'
#     )

##### adding the setting information
The file format was in mp4. Additioanally, we needed the speaker identification 
        for the transcription, so I added the information in the setting section

#### Maryam Code

In [ ]:
#ordinary
transcribe = boto3.client('transcribe')
all_jobs = []

# for i in range(len(file)):
now = datetime.now()
time_now = now.strftime('%H.%M.%S')
job_name = f'transcribe-jid-dual_channel_recording.m4a-{time_now}'
all_jobs.append(job_name)
# media_uri = f's3://{bucket}/{input_folder}/{file[i]}'
    
transcribe.start_transcription_job(
    TranscriptionJobName = job_name,
    MediaFormat = 'mp4',
    Media = {'MediaFileUri': f"s3://vnsny-research-transcribeinput-test/amazon-grant/amazon_grant-piolot-test/dual_channel_recording.m4a"},
    LanguageCode = 'en-US',
    OutputBucketName = bucket,
    OutputKey = f'{output_folder}/',
    # format of the output
    Settings={
        'ShowSpeakerLabels': True,           # clarify the label of the speaker
        'MaxSpeakerLabels': 2,               # number of thesspeakers in the file
        'ChannelIdentification': False,      # if the voice recorders has channel
        'ShowAlternatives': False,           # not sure about it
    }
)

In [38]:
len(all_jobs)

1

## Check job status

### Sherry's Code

In [39]:
# finish = False
# while not finish:
#     finish_status = [False] * len(file)
#     for i in range(len(file)):
#         response = transcribe.get_transcription_job(
#             TranscriptionJobName = all_jobs[i]
#         )
#         status = response['TranscriptionJob']['TranscriptionJobStatus']
#         if status in ['COMPLETED', 'FAILED']:
#             finish_status[i] = True
#             print(f'Job {all_jobs[i]} status is {status}')
#     if all(finish_status):
#         finish = True
#     else:
#         time.sleep(10)

#### Maryam code
Thanks for your codes for checking job status. It is great. I just simplified it. 

In [40]:
all_jobs[0]

'transcribe-jid-dual_channel_recording.m4a-10.22.16'

In [ ]:
#Ordinary
# for i in range(len(file)):
response = transcribe.get_transcription_job(
    TranscriptionJobName = all_jobs[0]
)
#     status = response['TranscriptionJob']['TranscriptionJobStatus']
#     if status in ['COMPLETED', 'FAILED']:
#         finish_status[0] = True
#         print(f'Job {all_jobs[0]} status is {status}')
print(response)

In [51]:
file = ['dual_channel_recording.m4a']

# Parse results

## Sherry's codes to extract content, cofidence, start_time, end_time

In [52]:
def get_transcript_text_and_timestamps(bucket_name, file_name):
    """take json file from S3 bucket and returns a tuple of:
       entire transcript, list object of tuples of timestamp and individual sentences
    
    Args:
        bucket_name (str): name of s3 bucket
        file_name (str): name of file
    Returns:
        (
        entire_transcript: str,
        sentences_and_times: [ {start_time (sec) : float,
                                end_time (sec)   : float,
                                sentence         : str,
                                min_confidence   : float (minimum confidence score of that sentence)
                                } ],
        confidences:  [ {start_time (sec) : float,
                         end_time (sec)   : float,
                         content          : str, (single word/phrase)
                         confidence       : float (confidence score of the word/phrase)
                         } ],
        scores: list of confidence scores
        )
    """
    s3_clientobj = s3.get_object(Bucket=bucket_name, Key=file_name)
    s3_clientdata = s3_clientobj["Body"].read().decode("utf-8")

    original = json.loads(s3_clientdata)
    print(original)
    items = original["results"]["items"]
    entire_transcript = original["results"]["transcripts"]

    sentences_and_times = []
    temp_sentence = ""
    temp_start_time = 0
    temp_min_confidence = 1.0
    newSentence = True
    
    confidences = []
    scores = []

    i = 0
    for item in items:
#         print("item:", item)
        # always add the word
        if item["type"] == "punctuation":
            temp_sentence = (
                temp_sentence.strip() + item["alternatives"][0]["content"] + " "
            )
        else:
            temp_sentence = temp_sentence + item["alternatives"][0]["content"] + " "
            temp_min_confidence = min(temp_min_confidence,
                                      float(item["alternatives"][0]["confidence"]))
            confidences.append({"start_time": float(item["start_time"]),
                                "end_time": float(item["end_time"]),
                                "content": item["alternatives"][0]["content"],
                                "confidence": float(item["alternatives"][0]["confidence"])
                               })
            scores.append(float(item["alternatives"][0]["confidence"]))

        # if this is a new sentence, and it starts with a word, save the time
        if newSentence == True:
            if item["type"] == "pronunciation":
                temp_start_time = float(item["start_time"])
            newSentence = False
        # else, keep going until you hit a punctuation
        else:
            if (
                item["type"] == "punctuation"
                and item["alternatives"][0]["content"] != ","
            ):
                # end time of sentence is end_time of previous word
                end_time = items[i-1]["end_time"] if i-1 >= 0 else items[0]["end_time"]
                sentences_and_times.append(
                    {"start_time": temp_start_time,
                     "end_time": end_time,
                     "sentence": temp_sentence.strip(),
                     "min_confidence": temp_min_confidence
                    }
                )
                # reset the temp sentence and relevant variables
                newSentence = True
                temp_sentence = ""
                temp_min_confidence = 1.0
                
        i = i + 1
        
    sentences_and_times.append(
                    {"start_time": temp_start_time,
                     "end_time": confidences[-1]["end_time"],
                     "sentence": temp_sentence.strip(),
                     "min_confidence": temp_min_confidence
                    }
                )
    return entire_transcript, sentences_and_times, confidences, scores

In [53]:
all_jobs

['transcribe-jid-dual_channel_recording.m4a-10.22.16']

In [ ]:
#ordinary
for i in range(len(all_jobs)):
    print(f'Parsing {all_jobs[i]}.json')
#     file_name = f'{output_folder}/{all_jobs[i]}.json'
    file_name ='/'.join(response['TranscriptionJob']['Transcript']['TranscriptFileUri'].split('/')[4:])
    entire_transcript, sentences_and_times, confidences, scores = get_transcript_text_and_timestamps(bucket, file_name)
    with open(f"{file[i].split('.')[0]}-entire-transcript.json", 'w') as f:
        json.dump(entire_transcript, f)
    with open(f"{file[i].split('.')[0]}-sentencess.json", 'w') as f:
        json.dump(sentences_and_times, f)
    with open(f"{file[i].split('.')[0]}-words.json", 'w') as f:
        json.dump(confidences, f)
        
sentences_and_times

In [57]:
### Maryam code - create dataframe for sherry's output

In [2]:
path = "path_to_a_folder"

In [64]:
df = pd.DataFrame(data = sentences_and_times, columns=["start_time", "end_time", "min_confidence", "sentence"])
df.to_excel(path + "dual_channel_sherry.xlsx")

## Maryam code to extract speakers' labels, content, confidence, start_time and end_time.

In [58]:
def get_transcript_text(bucket_name, file_name):

    s3_clientobj = s3.get_object(Bucket=bucket_name, Key=file_name)
    s3_clientdata = s3_clientobj["Body"].read().decode("utf-8")

    original = json.loads(s3_clientdata)
    items_spk = original["results"]['speaker_labels']["segments"]
    items_content = original["results"]["items"]
    entire_transcript = original["results"]["transcripts"]
    spk = []
    for i in range(len(items_spk)):
        for j in range(len(items_spk[i]['items'])):
            spk.append(items_spk[i]['items'][j]['speaker_label'])
    
    items_cont = []
    for i in range(len(items_content)):
        items_content[i]["content"] = items_content[i]["alternatives"][0]["content"]
        items_content[i]["confidence"] = items_content[i]["alternatives"][0]["confidence"]
        items_content[i].pop("alternatives", None)
        if items_content[i]["type"] == "punctuation":
            items_content[i-1]['content'] = items_content[i-1]['content'] + items_content[i]['content']
            continue
        items_cont.append(items_content[i])
        
    for i in range(len(items_cont)):
        items_cont[i].pop("type", None)
        items_cont[i]["speaker_label"] = spk[i]
    
    return items_cont

In [67]:
sent_time_spk = get_transcript_text(bucket, file_name)
df_data = pd.DataFrame(data=sent_time_spk, columns=sent_time_spk[0].keys())
df_data

,start_time,end_time,content,confidence,speaker_label
0,0.04,0.27,I'm,0.999,spk_1
1,0.27,0.75,okay.,0.787,spk_1
2,0.76,1.14,Thank,1.0,spk_1
3,1.14,1.24,you,1.0,spk_1
4,1.24,1.48,very,1.0,spk_1
...,...,...,...,...,...
2601,1132.38,1132.63,because,0.968,spk_0
2602,1132.63,1132.76,the,0.96,spk_0
2603,1132.76,1133.02,weather,1.0,spk_0
2604,1133.02,1133.2,was,0.986,spk_0


In [66]:
df_data.to_excel(path + "dual_channel_mary.xlsx")

## Clip using scipy

#### Converting mp4 to wav file for processing

In [7]:
from pydub import AudioSegment
import numpy as np
m4afile = AudioSegment.from_file(path+"dual_channel_recording.m4a")
np.array(m4afile.get_array_of_samples()).shape
# m4afile.export(path+"dual_c_r.wav", format='wav')

(50207744,)

In [ ]:
import scipy.io.wavfile
import numpy


# file_name = f'{file[-1]}'
# s3.download_file(Bucket=bucket, Key=f'{input_folder}/{file[-1]}', Filename=file_name)

rate, data = scipy.io.wavfile.read(path+"dual_c_r.wav")
edit_point = numpy.array([[860.1, 862.87],[880.86, 883.45]])  # "Most important, decrease the cigarette.", "I'll speak to you on Wednesday."
edit_point = numpy.round(edit_point*rate)

new_wav_file_list = []
for elem in edit_point:
    start = int(elem[0])
    end = int(elem[1])
    new_wav_file_list.extend(data[start:end])
    
new_wav_file = numpy.array(new_wav_file_list)
scipy.io.wavfile.write(path+'clipped.wav', rate, new_wav_file)

In [12]:
# import scipy.io.wavfile
# import numpy



# file_name = f'{file[-1]}'
# s3.download_file(Bucket=bucket, Key=f'{input_folder}/{file[-1]}', Filename=file_name)

# rate, data = scipy.io.wavfile.read(file_name)
# edit_point = numpy.array([[860.1, 862.87],[880.86, 883.45]])  # "Most important, decrease the cigarette.", "I'll speak to you on Wednesday."
# edit_point = numpy.round(edit_point*rate)

# new_wav_file_list = []
# for elem in edit_point:
#     start = int(elem[0])
#     end = int(elem[1])
#     new_wav_file_list.extend(data[start:end])
    
# new_wav_file = numpy.array(new_wav_file_list)
# scipy.io.wavfile.write('clipped.wav', rate, new_wav_file)